## Github 주소

https://github.com/KimSeokyeung/RS

## Assignment FINAL

## 데이터 불러오기

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
from collections import defaultdict

import numpy as np
import pandas as pd
import os

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

In [2]:
file_path=os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')
reader=Reader(line_format='user item rating timestamp',sep='\t')
data=Dataset.load_from_file(file_path,reader=reader)
trainset=data.build_full_trainset()

df=pd.DataFrame(data.raw_ratings,columns=["uid","iid","rate",'timestamp'])

## 데이터 전처리

In [3]:
list_id=df['uid'].value_counts().sort_values(ascending=False).index[:100] #rank 되어있는 것이 많은 user 100개
df=df[df['uid'].isin(list_id)]
list_item=df['iid'].value_counts().sort_values(ascending=False).index[:100] #rank 되어있는 것이 많은 item 100개
df=df[df['iid'].isin(list_item)]

df['uid'] = df['uid'].astype(int)
df['iid'] = df['iid'].astype(int)
df

,uid,iid,rate,timestamp
22,299,144,4.0,877881320
23,291,118,2.0,874833878
24,308,1,4.0,887736532
25,95,546,2.0,879196566
31,301,98,4.0,882075827
54,59,196,5.0,888205088
84,291,144,5.0,874835091
102,286,208,4.0,877531942
107,271,132,5.0,885848672
125,194,385,2.0,879524643


## Singularity 

In [4]:
#### R_1, R_2 ####
d = 2
R_1 = []
R_2 = []

d_max = int(max(df['rate']))
d_min = int(min(df['rate']))

for i in range(d_min, d_max+1):
    if(i<=d_max/d+1):
        R_1.append(i)
    else:
        R_2.append(i)

In [5]:
R_1

[1, 2, 3]

In [6]:
R_2

[4, 5]

In [7]:
#### P_1, P_2 ####
P_1 = {}
P_2 = {}

d_item = list(df['iid'].values)

for key in d_item:
    d_user = list(df[(df['iid']==key)&(df['rate'].isin(R_1))]['uid'].values)
    P_1[key] = d_user
    
for key in d_item:
    d_user = list(df[(df['iid']==key)&(df['rate'].isin(R_2))]['uid'].values)
    P_2[key] = d_user

In [8]:
P_1

{144: [280,
  308,
  94,
  387,
  286,
  385,
  437,
  417,
  393,
  339,
  716,
  244,
  666,
  399,
  655,
  932,
  314,
  234,
  682,
  916,
  653,
  561,
  406,
  650,
  805],
 118: [291,
  311,
  94,
  303,
  1,
  276,
  437,
  7,
  280,
  271,
  151,
  308,
  504,
  435,
  194,
  181,
  399,
  184,
  201,
  429,
  328,
  592,
  716,
  244,
  389,
  886,
  880,
  145,
  666,
  299,
  416,
  896,
  655,
  758,
  450,
  92,
  833,
  788,
  805,
  916,
  642,
  653,
  749],
 1: [181,
  268,
  145,
  234,
  279,
  271,
  13,
  561,
  363,
  59,
  299,
  429,
  655,
  650,
  788,
  727,
  796,
  883,
  293,
  537,
  201,
  889,
  393],
 546: [95,
  178,
  378,
  328,
  13,
  94,
  393,
  363,
  299,
  286,
  416,
  151,
  399,
  457,
  222,
  194,
  291,
  234,
  293,
  201,
  533,
  429,
  271,
  327,
  276,
  682,
  279,
  650,
  417,
  804,
  833,
  145,
  92,
  758,
  716,
  880,
  896,
  805,
  727,
  308,
  788,
  561,
  181,
  653,
  749,
  886,
  551,
  916,
  303],
 98: [308, 

In [9]:
P_2

{144: [299,
  291,
  279,
  13,
  328,
  293,
  327,
  222,
  363,
  194,
  92,
  450,
  429,
  301,
  374,
  416,
  313,
  276,
  378,
  457,
  1,
  95,
  178,
  592,
  551,
  749,
  303,
  201,
  883,
  896,
  833,
  435,
  758,
  864,
  804,
  788,
  880,
  268,
  889,
  7,
  796,
  130,
  886,
  497,
  727,
  648],
 118: [222,
  13,
  59,
  472,
  301,
  457,
  313,
  417,
  533,
  130,
  374,
  551,
  497,
  648,
  393,
  524,
  798,
  881,
  378,
  650,
  727,
  532,
  796,
  864,
  804,
  178],
 1: [308,
  280,
  95,
  194,
  94,
  311,
  286,
  301,
  18,
  222,
  374,
  339,
  327,
  450,
  472,
  417,
  313,
  1,
  416,
  291,
  533,
  592,
  532,
  184,
  406,
  151,
  642,
  244,
  749,
  276,
  92,
  716,
  178,
  798,
  648,
  497,
  378,
  435,
  804,
  130,
  886,
  303,
  896,
  864,
  805,
  314,
  916,
  387,
  389,
  399,
  870,
  653,
  881,
  894,
  457,
  880,
  932,
  682,
  747],
 546: [130,
  268,
  280,
  504,
  435,
  592,
  301,
  314,
  339,
  524,
  313,


In [10]:
#### S_1, S_2 ####

list_S1 = []
list_S2 = []

for key in P_1.keys():
    list_S1.append(len(P_1[key]))
    
for key in P_2.keys():
    list_S2.append(len(P_2[key]))
    
S = pd.DataFrame()
S['item'] = P_1.keys()
S['S_1'] = list_S1
S['S_2'] = list_S2
S['S_2'] = S['S_2']/len(df['uid'].unique())
S['S_1'] = S['S_1']/len(df['uid'].unique())

In [11]:
S

,item,S_1,S_2
0,144,0.25,0.46
1,118,0.43,0.26
2,1,0.23,0.59
3,546,0.49,0.21
4,98,0.08,0.82
5,196,0.22,0.57
6,208,0.19,0.53
7,132,0.21,0.57
8,385,0.34,0.42
9,443,0.24,0.42


In [12]:
#### A, G ####
def make_A_G(x,y):
    A_1_1 = []
    A_1_2 = []
    A_2_1 = []
    A_2_2 = []
    A = []
    
    for key in P_1.keys():
        if x in P_1[key]:
            if y in P_1[key]:
                A_1_1.append(key)
            elif y in P_2[key]:
                A_1_2.append(key)
    
    for key in P_1.keys():
        if x in P_2[key]:
            if y in P_1[key]:
                A_2_1.append(key)
            elif y in P_2[key]:
                A_2_2.append(key)

    A = [A_1_1, A_1_2, A_2_1, A_2_2]
    G = 0
    for i in range(len(A)):
        if A[i]:
            G += 1
    
    return A,G

In [13]:
#### SM 함수 ####
def SM(x,y):
    A,G = make_A_G(x,y)
    SM = 0
    sig = 0
    cnt = 0 
    
    for j in range(0,2):
        sig = 0
        for q in range(0,2):
            A_use = A[cnt]
            A_use.sort()
            if A_use:
                r_x = df[(df['uid']==x)&(df['iid'].isin(A_use))].sort_values(by='iid')['rate'].values
                r_y = df[(df['uid']==y)&(df['iid'].isin(A_use))].sort_values(by='iid')['rate'].values
                s_j = S[S['item'].isin(A_use)].sort_values(by='item')['S_'+str(j+1)].values
                s_p = S[S['item'].isin(A_use)].sort_values(by='item')['S_'+str(q+1)].values
                sig = sum((1-(r_x-r_y)**2)*s_j*s_p)
                sig /= len(A_use) 
        SM += sig
        cnt+=1
    SM /= G
    
    return SM

In [14]:
SM_mat=np.zeros([100,100])
uid_list=df['uid'].sort_values().unique()

for i in range(0,100):
    for j in range(0,100):
        if i!=j:
            SM_mat[i][j] = SM(uid_list[i], uid_list[j])
        else:
            SM_mat[i][j] = 1

In [15]:
SM_mat

array([[ 1.        , -0.23079375, -0.07143896, ..., -0.11890644,
        -0.04748786, -0.05162   ],
       [-0.19678393,  1.        , -0.21244028, ..., -0.20323214,
        -0.07195   , -0.15389375],
       [-0.17884925, -0.27921869,  1.        , ..., -0.13292361,
        -0.11806561, -0.12906333],
       ...,
       [-0.1282725 , -0.20517688, -0.19481103, ...,  1.        ,
        -0.09991094, -0.20531786],
       [-0.05927526, -0.08679828, -0.08474926, ..., -0.03899616,
         1.        , -0.01460818],
       [-0.11837727, -0.20863864, -0.23241071, ..., -0.11680893,
        -0.177555  ,  1.        ]])

In [16]:
df_SM = pd.DataFrame(SM_mat, columns=uid_list, index=uid_list)
df_SM

,1,7,13,18,59,85,90,92,94,95,...,870,880,881,883,886,889,894,896,916,932
1,1.000000,-0.230794,-0.071439,-0.046544,-0.135981,-0.063301,-0.244454,-0.116003,-0.119973,-0.088241,...,-0.027757,-0.012902,-0.159606,-0.110825,-0.217391,-0.118605,-0.134113,-0.118906,-0.047488,-0.051620
7,-0.196784,1.000000,-0.212440,-0.210475,-0.177838,-0.106994,-0.245369,-0.084019,-0.102333,-0.094412,...,-0.125592,-0.139687,-0.045412,-0.007006,-0.146406,-0.091338,-0.031900,-0.203232,-0.071950,-0.153894
13,-0.178849,-0.279219,1.000000,-0.138332,-0.267289,-0.054824,-0.235249,-0.160390,-0.226104,-0.224662,...,-0.114855,-0.125161,-0.200274,-0.138413,-0.265487,-0.155519,-0.229410,-0.132924,-0.118066,-0.129063
18,-0.120193,-0.129355,-0.137437,1.000000,-0.165747,0.015425,-0.129331,-0.035495,-0.032708,-0.068462,...,0.019667,-0.096377,-0.048909,-0.090076,-0.060713,-0.005165,0.000000,-0.111939,-0.024890,-0.110102
59,-0.169627,-0.144869,-0.115487,-0.222490,1.000000,-0.055742,-0.213685,-0.124020,-0.105304,-0.256919,...,-0.224300,-0.074578,-0.167601,0.034628,-0.141835,-0.064443,-0.050413,-0.139934,-0.087356,-0.093144
85,-0.141524,-0.172997,-0.115996,-0.159624,-0.166085,1.000000,-0.147695,-0.102763,-0.100097,-0.156390,...,-0.011212,-0.045507,-0.059870,-0.068549,-0.084339,-0.044256,0.009908,-0.078415,-0.073885,-0.048853
90,-0.173452,-0.235386,-0.214818,-0.021231,-0.345429,-0.188760,1.000000,-0.188146,-0.209471,-0.231696,...,-0.202306,-0.243343,-0.079990,-0.106667,0.013250,-0.028308,0.038500,-0.483769,-0.703717,-0.210675
92,-0.105047,-0.165469,-0.112491,-0.156671,-0.171503,-0.043127,-0.149556,1.000000,-0.073063,-0.053964,...,-0.079156,-0.083611,-0.086005,-0.047842,-0.183816,-0.074840,-0.862200,-0.076992,-0.031343,-0.011429
94,-0.117835,-0.144892,-0.128613,-0.072458,-0.105644,0.017716,-0.189793,-0.119345,1.000000,-0.108655,...,-0.030074,-0.034953,-0.056470,-0.106378,-0.045967,-0.010890,0.021325,-0.105652,-0.052877,-0.045909
95,-0.069224,-0.193720,-0.176000,-0.073515,-0.189300,-0.101815,-0.171020,-0.137733,-0.159032,1.000000,...,-0.086281,-0.216899,-0.091843,-0.201886,-0.082534,-0.042530,-0.153000,-0.142161,-0.088173,-0.108662


## PIP

In [17]:
def Agreement(r1,r2):
    R_max=5
    R_min=1
    
    R_med=(R_max+R_min)/2
    
    if ((r1-R_med)*(r2-R_med)>0): #같을때
        return True
    else:
        return False

def Distance(r1,r2):
    D=Agreement(r1,r2)
    
    if (D==True):
        return abs(r1-r2)
    else:
        return 2*abs(r1-r2)

In [18]:
def Impact(u1,u2):
    
    R_max=5
    R_min=1
    R_med=(R_max+R_min)/2
    
    Im=[]
    iid_u1=df[df['uid']==u1]['iid']
    iid_u2=df[df['uid']==u2]['iid']
    iid_u1=iid_u1.unique()
    iid_u2=iid_u2.unique()
    iid_merge=list(set(iid_u1).intersection(iid_u2))
    
    for i in iid_merge:
        r1=int(df[(df['uid']==u1) & (df['iid']==i)].rate)
        r2=int(df[(df['uid']==u2) & (df['iid']==i)].rate)
        R=Agreement(r1,r2)

        if (R==True):
            I=(abs(r1-R_med)+1)*(abs(r2-R_med)+1)
            Im.append(I)
        else:
            I=1/((abs(r1-R_med)+1)*abs(r2-R_med)+1)
            Im.append(I)
            
    return Im

In [19]:
def Proximity(u1,u2):
    R_max=5
    R_min=1
    
    pro=[]
    iid_u1=df[df['uid']==u1]['iid']
    iid_u2=df[df['uid']==u2]['iid']
    iid_u1=iid_u1.unique()
    iid_u2=iid_u2.unique()
    iid_merge=list(set(iid_u1).intersection(iid_u2))
    
    for i in iid_merge:
        r1=int(df[(df['uid']==u1) & (df['iid']==i)].rate)
        r2=int(df[(df['uid']==u2) & (df['iid']==i)].rate)
    
        D=Distance(r1,r2)
        Pro=(2*(R_max-R_min)+1-D)**2
        pro.append(Pro)
        
    return pro

In [20]:
def Popularity(u1,u2):
    ##popularity
    popularity=[]
    iid_u1=df[df['uid']==u1]['iid']
    iid_u2=df[df['uid']==u2]['iid']
    iid_u1=iid_u1.unique()
    iid_u2=iid_u2.unique()
    iid_merge=list(set(iid_u1).intersection(iid_u2))
    
    iid_mean=pd.DataFrame(df.groupby('iid')['rate'].mean())
    iid_mean=iid_mean[iid_mean.index.isin(iid_merge)]

    
    for i in iid_merge:
        pop=0
        r1=int(df[(df['uid']==u1) & (df['iid']==i)].rate)
        r2=int(df[(df['uid']==u2) & (df['iid']==i)].rate)

        test=((r1-float(iid_mean.loc[i]))*(r2-float(iid_mean.loc[i])))
        
        if (test > 0) :
            pop = 1+(((r1+r2)/2)-float(iid_mean.loc[i]))**2
            popularity.append(pop)
        else :
            popularity.append(1)

    return popularity

In [21]:
def PIP(u1,u2):
    
    iid_u1=df[df['uid']==u1]['iid']
    iid_u2=df[df['uid']==u2]['iid']
    iid_u1=iid_u1.unique()
    iid_u2=iid_u2.unique()
    iid_merge=list(set(iid_u1).intersection(iid_u2))
    
    pro = np.array(Proximity(u1,u2))
    Im = np.array(Impact(u1,u2))
    popul = np.array(Popularity(u1,u2))
    
    PIP = pro*Im*popul
    
    df_PIP = pd.DataFrame(PIP, columns={'PIP'}, index=iid_merge)
    
    return df_PIP

In [22]:
PIP(301,291)

,PIP
385,24.500000
1,526.772159
4,388.000000
7,528.060000
9,8.333333
393,88.042899
11,326.748615
12,384.000000
143,49.000000
144,578.266217
